In [ ]:
# !pip install transformers datasets evaluate
# or
# !conda install transformers datasets evaluate

In [43]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

imdb = load_dataset("imdb")


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Found cached dataset imdb (/Users/ericchantland/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [45]:
x = imdb["train"][0]['text']
x

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [53]:

import re
 
# test_str = 'Gfg, is best: for ! Geeks ;'
x = imdb["train"][0]['text']

res = re.sub(r'[^\w\s]', ' ', x)
print(res)

I rented I AM CURIOUS YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967  I also heard that at first it was seized by U S  customs if it ever tried to enter this country  therefore being a fan of films considered  controversial  I really had to see this for myself  br    br   The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life  In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States  In between asking politicians and ordinary denizens of Stockholm about their opinions on politics  she has sex with her drama teacher  classmates  and married men  br    br   What kills me about I AM CURIOUS YELLOW is that 40 years ago  this was considered pornographic  Really  the sex and nudity scenes are few and far between  eve

In [76]:
word_count = set()
for words in imdb["train"]:
    words_split = re.sub(r'[^\w\s]', ' ', words['text'])
    words_split = set(words_split.split())
    word_count.update(words_split)

In [77]:
len(word_count)

93929

## Preprocess
<br>
The next step is to load a DistilBERT tokenizer to preprocess the text field:

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Create a preprocessing function to tokenize text and truncate sequences to be no longer than DistilBERT’s maximum input length:

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

To apply the preprocessing function over the entire dataset, use 🤗 Datasets map function. You can speed up map by setting batched=True to process multiple elements of the dataset at once:

In [8]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Loading cached processed dataset at /Users/ericchantland/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-bfa4e62ed21c92aa.arrow
Loading cached processed dataset at /Users/ericchantland/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-fb62cccaac8fd0aa.arrow


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [21]:
imdb['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [79]:
# tokenized_imdb['train'][0]

In [36]:
max_len = 0
phrase = ""
sequence_i = 0
for i, tx in enumerate(tokenized_imdb['train']):
    if len(tx['input_ids']) > max_len:
        max_len = len(tx['input_ids'])
        phrase = tx['text']
        sequence_i = i
print(max_len)
print(phrase)
print(sequence_i)

512
Who are these "They"- the actors? the filmmakers? Certainly couldn't be the audience- this is among the most air-puffed productions in existence. It's the kind of movie that looks like it was a lot of fun to shoot TOO much fun, nobody is getting any actual work done, and that almost always makes for a movie that's no fun to watch.<br /><br />Ritter dons glasses so as to hammer home his character's status as a sort of doppleganger of the bespectacled Bogdanovich; the scenes with the breezy Ms. Stratten are sweet, but have an embarrassing, look-guys-I'm-dating-the-prom-queen feel to them. Ben Gazzara sports his usual cat's-got-canary grin in a futile attempt to elevate the meager plot, which requires him to pursue Audrey Hepburn with all the interest of a narcoleptic at an insomnia clinic. In the meantime, the budding couple's respective children (nepotism alert: Bogdanovich's daughters) spew cute and pick up some fairly disturbing pointers on 'love' while observing their parents. (

In [38]:

phrase = []
sequence_i = []
for i, tx in enumerate(tokenized_imdb['train']):
    if len(tx['input_ids']) == 512:
        phrase.append(tx['text'])
        sequence_i.append(i)
print(len(phrase))
print(sequence_i)

3715
[8, 17, 34, 42, 44, 52, 55, 69, 74, 80, 87, 92, 95, 98, 111, 112, 128, 142, 147, 149, 165, 180, 197, 198, 209, 213, 218, 230, 238, 246, 248, 253, 257, 264, 265, 273, 281, 282, 298, 312, 345, 353, 370, 374, 375, 394, 413, 415, 416, 421, 422, 440, 457, 459, 461, 478, 479, 483, 498, 500, 501, 509, 517, 520, 521, 528, 531, 543, 581, 591, 593, 603, 612, 614, 615, 618, 651, 652, 655, 657, 661, 666, 667, 668, 674, 681, 687, 688, 689, 700, 706, 725, 726, 735, 736, 740, 745, 746, 749, 750, 751, 753, 757, 759, 762, 768, 771, 773, 779, 784, 788, 797, 803, 812, 814, 825, 831, 832, 848, 877, 878, 896, 899, 913, 922, 926, 932, 942, 958, 965, 974, 982, 983, 984, 989, 1001, 1019, 1020, 1025, 1026, 1041, 1056, 1075, 1080, 1084, 1090, 1104, 1107, 1109, 1123, 1127, 1131, 1147, 1167, 1173, 1174, 1175, 1181, 1197, 1199, 1212, 1213, 1214, 1216, 1228, 1233, 1244, 1245, 1261, 1263, 1267, 1281, 1282, 1306, 1330, 1332, 1336, 1340, 1342, 1347, 1361, 1364, 1371, 1391, 1395, 1400, 1406, 1427, 1432, 1439, 1444

Now create a batch of examples using <a href="https://huggingface.co/docs/transformers/v4.29.0/en/main_classes/data_collator#transformers.DataCollatorWithPadding"> DataCollatorWithPadding</a>. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate <br>
Including a metric during training is often helpful for evaluating your model’s performance. You can quickly load a evaluation method with the 🤗 Evaluate library. For this task, load the accuracy metric (see the 🤗 Evaluate quick tour to learn more about how to load and compute a metric):

In [10]:
import evaluate

accuracy = evaluate.load("accuracy")

Then create a function that passes your predictions and labels to compute to calculate the accuracy:

In [11]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Your compute_metrics function is ready to go now, and you’ll return to it when you setup your training.


## Train
Before you start training your model, create a map of the expected ids to their labels with id2label and label2id:

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

### Pytorch

If you aren’t familiar with finetuning a model with the Trainer, take a look at the basic tutorial <a href="https://huggingface.co/docs/transformers/training#train-with-pytorch-trainer">here</a>!

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

## Inference

In [3]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="Chantland/my_awesome_model")

In [4]:
# text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
text = "Butter my toast, this was bad!"
classifier(text)

[{'label': 'NEGATIVE', 'score': 0.9917591214179993}]

### Manual inference via Pytorch

(This is like the above inference, only manual)
Tokenize the text and return PyTorch tensors:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_model")
inputs = tokenizer(text, return_tensors="pt")

Pass your inputs to the model and return the logits:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("stevhliu/my_awesome_model")
with torch.no_grad():
    logits = model(**inputs).logits

Get the class with the highest probability, and use the model’s id2label mapping to convert it to a text label:

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

## Tokenizing practice

In [4]:
import numpy as np
y_train = np.random.randint(0,2,3)
x_train = ["agsf","fuhaseuif", "jwfhaiu"]
y_val = np.random.randint(0,2,3)
x_val = ["agsf","fuhaseuif", "jwfhaiu"]
y_test = np.random.randint(0,2,3)
x_test = ["agsf","fuhaseuif", "jwfhaiu"]

In [9]:
d = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),
     'val':Dataset.from_dict({'label':y_val,'text':x_val}),
     'test':Dataset.from_dict({'label':y_test,'text':x_test})
     })

In [10]:
d["train"][0]

{'label': 1, 'text': 'agsf'}

In [11]:
d

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 3
    })
    val: Dataset({
        features: ['label', 'text'],
        num_rows: 3
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 3
    })
})